# Sample workflow (Wyoming) based on [Jonathon's document]()

## defined classes and functions in `nhgisxwalk` package


#### notes -- to do

* single function or make class?
* nhgisify -- G + 0 + 0
* geoid <--> nhgisid
* `geography_codes(geog="block")` -- code label to code description xwalk

#### NHGIS [block crosswalks](https://www.nhgis.org/user-resources/geographic-crosswalks)


In [1]:
%load_ext watermark
%watermark

2020-04-29T09:37:26-04:00

CPython 3.7.6
IPython 7.13.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [2]:
import nhgisxwalk
import glob
import inspect
import numpy
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
pandas     1.0.3
numpy      1.18.1
nhgisxwalk 0.0.0



## (major) Defined functions

In [3]:
print(inspect.getsource(nhgisxwalk.calculate_atoms))

def calculate_atoms(
    df, weight=None, input_var=None, sum_var=None, groupby_cols=None
):
    """ Calculate the atoms (intersecting parts) of census geographies.
    
    Parameters
    ----------
    
    df : pandas.DataFrame
        input data
    
    weight : str
        weight colum name
    
    input_var : str
        input variable column name
    
    sum_var : str
        groupby and summed variable column name
    
    groupby_cols : list
        dataframe columns to perform groupby
    
    Returns
    -------
    
    atoms : pandas.DataFrame
        output data
    
    """
    
    df[sum_var] = df[weight] * df[input_var]
    
    atoms = df.groupby(groupby_cols)[sum_var].sum().to_frame()
    
    atoms.reset_index(inplace=True)
    
    return atoms



In [4]:
print(inspect.getsource(nhgisxwalk.bgp_id))

def bgp_id(
    df, order, cname="_GJOIN", tzero=["STATEA", "COUNTYA"], nhgis=True
):
    """recreate a the BGPs GISJOIN/GEOID
    
    Parameters
    ----------
    
    df : pandas.DataFrame
        Input dataframe.
    
    order : list-like
        Correct ordering of columns.
    
    cname : str
        Name for new column.
    
    tzeros : list
        Columns to add trailing zero. `nhgis` must be True.
    
    nhgis : bool
        Added 'G' and training zeros. See `GISJOIN identifiers` at
        https://www.nhgis.org/user-resources/geographic-crosswalks
    
    Returns
    -------
    
    df : pandas.DataFrame
        Dataframe with new column.
    
    """
    
    def _gjoin(x):
        """internal GISJOIN/GEOID generator"""
        
        # container for ID components
        join_id_vals = []
        for o in order:
            
            # if the val associated with `o` is a numpy.float
            v = getattr(x, o)
            
            try:
                # 

In [5]:
print(inspect.getsource(nhgisxwalk.trt_id))

def trt_id(year, _id, nhgis=True):
    """Extract the tract ID from the block ID.
    
    Parameters
    ----------
    
    year : str
        Census collection year.
    
    _id : str
        Block GISJOIN/GEOID.
    
    nhgis : bool
        Added 'G' and training zeros. See `GISJOIN identifiers` at
        https://www.nhgis.org/user-resources/geographic-crosswalks
    
    Returns
    -------
    
    tract_id : str
        Tract GISJOIN/GEOID.
    
    """
    
    if year == "2010":
        indexer = 14
        if not nhgis:
            indexer = "?"
        # slice out tract ID
        tract_id = _id[:indexer]
    else:
        msg = "Census year %s is not currently supported." % year
        raise RuntimeError(msg)
    
    return tract_id



In [6]:
print(inspect.getsource(nhgisxwalk.id_from))

def id_from(target_func, target_year, source, vectorized=True):
    """Create target IDs from source IDs.
    
    Parameters
    ----------
    
    target_func : function or method
        function or method for generating requested target IDs
    
    target_year : str
        target ID year
    
    source : iterable
        original source IDs
    
    vectorized : bool
        potential speedup when vectorized
    
    Returns
    -------
    
    result : iterable
        generated target IDs
    
    """
    
    # generate IDs from source geographies to target geographies
    if vectorized:
        result = numpy.vectorize(target_func)("2010", source)
    else:
        result = [target_func("2010", rec) for rec in source]
    
    return result



------------------

## Step 1
### 1990-2010 BLK Crosswalk (`GISJOIN`)


In [7]:
xwalk_file = "crosswalks/nhgis_blk1990_blk2010_gj.csv.zip"
data_types = nhgisxwalk.str_types(["GJOIN1990", "GJOIN2010"])
data_types

{'GJOIN1990': str, 'GJOIN2010': str}

In [8]:
blk_xwdf = pandas.read_csv(xwalk_file, index_col=0, dtype=data_types)
blk_xwdf.head()

/Users/jgaboardi/miniconda3/envs/nhgis_env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,GJOIN1990,GJOIN2010,WEIGHT,PAREA_VIA_BLK00
0,NaN,G01000300107014085,0.0,0.0
1,NaN,G01000300107014086,0.0,0.0
2,NaN,G01000300107014089,0.0,0.0
3,NaN,G01000300107014091,0.0,0.0
4,NaN,G01000300107014109,0.0,0.0


#### We can ignore the `NaN` 1990 values (which don't occur in WY) then susbet to WY

In [9]:
blk_xwdf = blk_xwdf[~blk_xwdf["GJOIN1990"].isna()]
blk_xwdf = blk_xwdf[blk_xwdf["GJOIN1990"].map(lambda x: x[1:3] == "56")]
blk_xwdf.shape

(220403, 4)

In [10]:
blk_xwdf.head()

,GJOIN1990,GJOIN2010,WEIGHT,PAREA_VIA_BLK00
20066154,G56000109626101,G56000109639001000,0.977864,0.977864
20066155,G56000109626101,G56000109639001001,0.013341,0.000193
20066156,G56000109626101,G56000109639001002,0.000191,0.000191
20066157,G56000109626101,G56000109639001009,0.008020,0.008020
20066158,G56000109626101,G56000109639001017,0.000274,0.013423


------------------------
## Step 2
### 1990 Blocks (Tabular)

In [11]:
csvs = glob.glob("tabular_data/WY_1990_block/*.csv")
csvs

['tabular_data/WY_1990_block/1990_block.csv',
 'tabular_data/WY_1990_block/context_cookbook_1990_BLOCK.csv']

In [12]:
context = nhgisxwalk.get_context("Block-1990", csvs[1])
context

,Variable,Description
Block-1990,,
0,GISJOIN,GIS Join Match Code
1,YEAR,Data File Year
2,ANRCA,Alaska Native Regional Corporation Code
3,AIANHHA,American Indian Area/Alaska Native Area/H...
4,RES_ONLYA,American Indian Reservation [excluding trus...
5,TRUSTA,American Indian Reservation [trust lands...
6,RES_TRSTA,Reservation/Trust Lands Code
7,BLOCKA,Block Code
8,BLCK_GRPA,Block Group Code


In [13]:
data_types = nhgisxwalk.str_types(context["Variable"])
tabblkdf = pandas.read_csv(csvs[0], dtype=data_types)
tabblkdf.columns

Index(['GISJOIN', 'YEAR', 'ANRCA', 'AIANHHA', 'RES_ONLYA', 'TRUSTA',
       'RES_TRSTA', 'BLOCKA', 'BLCK_GRPA', 'TRACTA', 'CDA', 'C_CITYA',
       'COUNTY', 'COUNTYA', 'CTY_SUBA', 'DIVISIONA', 'MSA_CMSAA', 'PLACEA',
       'PMSAA', 'REGIONA', 'STATE', 'STATEA', 'URBRURALA', 'URB_AREAA',
       'CD103A', 'ANPSADPI', 'ET1001', 'EUY001', 'EUY002', 'EUY003', 'EUY004',
       'EUY005', 'ESA001', 'ES1001', 'ES1002'],
      dtype='object')

In [14]:
tabblkdf.head()

,GISJOIN,YEAR,ANRCA,AIANHHA,RES_ONLYA,TRUSTA,RES_TRSTA,BLOCKA,BLCK_GRPA,TRACTA,...,ANPSADPI,ET1001,EUY001,EUY002,EUY003,EUY004,EUY005,ESA001,ES1001,ES1002
0,G56000109626102,1990,99,9999,9999,9999,9,102,1,9626,...,Block 102,4,4,0,0,0,0,4,0,1
1,G56000109626103,1990,99,9999,9999,9999,9,103,1,9626,...,Block 103,26,26,0,0,0,0,72,8,2
2,G56000109626105,1990,99,9999,9999,9999,9,105,1,9626,...,Block 105,2,2,0,0,0,0,1,1,0
3,G56000109626106,1990,99,9999,9999,9999,9,106,1,9626,...,Block 106,0,0,0,0,0,0,14,0,0
4,G56000109626107,1990,99,9999,9999,9999,9,107,1,9626,...,Block 107,2,2,0,0,0,0,28,1,0


### Join Block Table and Crosswalk

In [15]:
blk_df = pandas.merge(
    left=blk_xwdf,
    right=tabblkdf,
    how="left",
    left_on="GJOIN1990",
    right_on="GISJOIN",
    validate="many_to_many"
)

### Variables

In [16]:
id_and_weight = [
    "GJOIN1990",
    "GISJOIN",
    "WEIGHT",
    "PAREA_VIA_BLK00"
]
var_cols = [
    _k for k,v in nhgisxwalk.code_desc_1990_blk.items()
    for _k, _ in v.items()
    if _k.startswith("E")
]
blk_df[id_and_weight + var_cols].head(10)

,GJOIN1990,GISJOIN,WEIGHT,PAREA_VIA_BLK00,ET1001,ET1001,EUY001,EUY002,EUY003,EUY004,EUY005,ET1001,ES1001,ES1002
0,G56000109626101,NaN,0.977864,0.977864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,G56000109626101,NaN,0.013341,0.000193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,G56000109626101,NaN,0.000191,0.000191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,G56000109626101,NaN,0.008020,0.008020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,G56000109626101,NaN,0.000274,0.013423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,G56000109626101,NaN,0.000310,0.000310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,G56000109626102,G56000109626102,0.279581,0.493676,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
7,G56000109626102,G56000109626102,0.000000,0.002995,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
8,G56000109626102,G56000109626102,0.000000,0.000531,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
9,G56000109626102,G56000109626102,0.000000,0.000152,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0


### We can safely assume `NaN` values after join are 0.0 due to blocks with no population or housing units being excluded in the 1990 Census -- see [here](https://hackmd.io/pr87vv2kQG6JibTZU3s1ZA?both#NaNs)

In [17]:
blk_df = blk_df[~blk_df["GISJOIN"].isna()]
blk_df.shape

(78532, 39)

In [18]:
blk_df[var_cols].head()

,ET1001,ET1001,EUY001,EUY002,EUY003,EUY004,EUY005,ET1001,ES1001,ES1002
6,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
7,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
8,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
9,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0
10,4.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,1.0


------------------

### Add Block Group Part ID to the blocks data

In [19]:
bgp_cols = nhgisxwalk.code_cols("bgp", "1990")
bgp_cols

['STATEA',
 'COUNTYA',
 'CTY_SUBA',
 'PLACEA',
 'TRACTA',
 'CDA',
 'AIANHHA',
 'RES_TRSTA',
 'ANRCA',
 'URB_AREAA',
 'URBRURALA',
 'BLCK_GRPA']

In [20]:
blk_df = nhgisxwalk.bgp_id(blk_df, bgp_cols, cname="bgp1990", nhgis=True)
blk_df[["GJOIN1990", "GJOIN1990", "bgp1990"]].head()

,GJOIN1990,GJOIN1990,bgp1990
6,G56000109626102,G56000109626102,G560001092695999999626009999999999921
7,G56000109626102,G56000109626102,G560001092695999999626009999999999921
8,G56000109626102,G56000109626102,G560001092695999999626009999999999921
9,G56000109626102,G56000109626102,G560001092695999999626009999999999921
10,G56000109626102,G56000109626102,G560001092695999999626009999999999921


------------------------
## Step 3 — Calculate base atoms

#### See [a synthetic example here](https://nbviewer.jupyter.org/gist/jGaboardi/9f032b7713179dec292def31fe34954d?flush_cache=true). For further background information see:
* **Schroeder, J. P**. 2007. *Target-density weighting interpolation and uncertainty evaluation for temporal analysis of census data*. Geographical Analysis 39 (3):311–335.

In [21]:
blk_df["trt2010"] = nhgisxwalk.id_from(
    nhgisxwalk.trt_id, "2010", blk_df["GJOIN2010"]
)

In [22]:
population_total = nhgisxwalk.desc_code_1990_blk["Persons"]["Total"]
xwcols = [
    "bgp1990", "GJOIN1990", "GJOIN2010", "trt2010", "WEIGHT", population_total
]
blk_df[xwcols].head()

,bgp1990,GJOIN1990,GJOIN2010,trt2010,WEIGHT,ET1001
6,G560001092695999999626009999999999921,G56000109626102,G56000109639001003,G5600010963900,0.279581,4.0
7,G560001092695999999626009999999999921,G56000109626102,G56000109639001004,G5600010963900,0.000000,4.0
8,G560001092695999999626009999999999921,G56000109626102,G56000109639001005,G5600010963900,0.000000,4.0
9,G560001092695999999626009999999999921,G56000109626102,G56000109639001006,G5600010963900,0.000000,4.0
10,G560001092695999999626009999999999921,G56000109626102,G56000109639001007,G5600010963900,0.000000,4.0


### Calculate atoms

In [23]:
atom_df = nhgisxwalk.calculate_atoms(
    blk_df,
    weight="WEIGHT",
    input_var=population_total,
    sum_var="bgp1990_in_trt2010",
    groupby_cols=["bgp1990", "trt2010"]
)
atom_df.head()

,bgp1990,trt2010,bgp1990_in_trt2010
0,G560001091100999999626009999999999922,G5600010963900,96.000000
1,G560001092090450509628009999999999911,G5600010962800,973.000000
2,G560001092090450509628009999999999912,G5600010962700,0.002397
3,G560001092090450509628009999999999912,G5600010962800,1059.997603
4,G560001092090450509629009999999999911,G5600010962900,838.000000


------------------------
## Step 4
### Join 1990 Block Group Part (Tabular)
#### BGP Join ID

In [24]:
# this is already performed earlier

------------------------
## Step 5
### Estimate black & OOHU counts for atoms

#### Atoms for subgroups

**These atoms of `y1y2` unit intersections can then be used to calculate subgroups of variables with the following equation:**
## $atom_{\widehat{var}^{gu}} = total_{\widehat{var}^{gu}} \times \frac{atom_{var^{gu}}}{total_{var^{gu}}}$
**where:**
 * $var =$ a variable representing a count (i.e. population)
 * $\widehat{var} =$ a subgroup of a variable representing a count (i.e. asian population)
 * $gu =$ a geographic unit (i.e. block group part)

**For example, since we are considering the population variable, we will define the following:**
## $rg_i = \widehat{var}$
**where**
 * $pop = $ total population variable
 * ${rg}_i =$ the population of a specific racial group

## $atom_{rg_i^{bgp}} = total_{rg_i^{bgp}} \times \frac{atom_{pop^{bgp}}}{total_{pop^{bgp}}}$

**As an example, consider the are 30 people in racial group $i$ within block group part $A$ $({total}_{rg_i^{bgp}})$:**
* ${atom}_{rg_i^{AX}} = 16.875 = 30 \times \frac{90}{160}$
* ${atom}_{rg_i^{AY}} = 13.125 = 30 \times \frac{70}{160}$

In [25]:
xwcols = ["bgp1990", "GJOIN1990", "GJOIN2010", "trt2010", "WEIGHT", "ET1001", "EUY002"]
blk_df[xwcols].head()

,bgp1990,GJOIN1990,GJOIN2010,trt2010,WEIGHT,ET1001,EUY002
6,G560001092695999999626009999999999921,G56000109626102,G56000109639001003,G5600010963900,0.279581,4.0,0.0
7,G560001092695999999626009999999999921,G56000109626102,G56000109639001004,G5600010963900,0.000000,4.0,0.0
8,G560001092695999999626009999999999921,G56000109626102,G56000109639001005,G5600010963900,0.000000,4.0,0.0
9,G560001092695999999626009999999999921,G56000109626102,G56000109639001006,G5600010963900,0.000000,4.0,0.0
10,G560001092695999999626009999999999921,G56000109626102,G56000109639001007,G5600010963900,0.000000,4.0,0.0


In [26]:
print(blk_df.groupby("bgp1990")["ET1001"].sum().shape)
blk_df.groupby("bgp1990")["ET1001"].sum().head()

(1074,)


bgp1990
G560001091100999999626009999999999922     1811.0
G560001092090450509628009999999999911     3335.0
G560001092090450509628009999999999912     2759.0
G560001092090450509629009999999999911    11447.0
G560001092090450509629009999999999912     1556.0
Name: ET1001, dtype: float64

In [27]:
blk_df.groupby("bgp1990")["bgp1990_in_trt2010"].sum().head()

bgp1990
G560001091100999999626009999999999922      96.0
G560001092090450509628009999999999911     973.0
G560001092090450509628009999999999912    1060.0
G560001092090450509629009999999999911     838.0
G560001092090450509629009999999999912     867.0
Name: bgp1990_in_trt2010, dtype: float64

In [28]:
blk_df.groupby("bgp1990")["EUY002"].sum().head()

bgp1990
G560001091100999999626009999999999922     0.0
G560001092090450509628009999999999911    12.0
G560001092090450509628009999999999912    11.0
G560001092090450509629009999999999911    72.0
G560001092090450509629009999999999912    26.0
Name: EUY002, dtype: float64

In [29]:
0. * (96. / 1811.)

0.0

In [30]:
12. * (973. / 3335.)

3.501049475262369

-----------------------------------------------